튜토리얼 영상 : https://youtu.be/cUoJDBFsJ8Y

1. 예제 데이터 만들기
병합 실습을 위해 공통된 컬럼(x1)을 가졌지만, 들어있는 값은 조금씩 다른 두 개의 데이터프레임(adf, bdf)을 만듭니다.

In [ ]:
import pandas as pd

In [ ]:
# 1. 왼쪽 데이터프레임 (adf) 생성
# x1 컬럼: A, B, C
# x2 컬럼: 1, 2, 3
adf = pd.DataFrame({"x1": ["A", "B", "C"], "x2": [1, 2, 3]})
adf

In [ ]:
# 2. 오른쪽 데이터프레임 (bdf) 생성
# x1 컬럼: A, B, D (C가 없고 D가 있음 -> 차이점 발생)
# x3 컬럼: T, F, T
bdf = pd.DataFrame({"x1": ["A", "B", "D"], "x3": ["T", "F", "T"]})
bdf

2. 병합 방식 (Merge Types)
pd.merge(left, right, how='...', on='key') 문법을 사용하여 다양한 방식으로 데이터를 합칩니다.

In [ ]:
# 3. Left Join (왼쪽 기준 병합)
# how='left': 왼쪽(adf)의 모든 데이터를 유지합니다.
# 오른쪽(bdf)에 없는 값(여기서는 'C')은 NaN(결측치)으로 채워집니다.
# 결과: A, B는 매칭되고 C는 x3가 NaN이 됨.
pd.merge(adf, bdf, how='left', on='x1')

In [ ]:
# 4. Right Join (오른쪽 기준 병합)
# how='right': 오른쪽(bdf)의 모든 데이터를 유지합니다.
# 왼쪽(adf)에 없는 값(여기서는 'D')은 NaN으로 채워집니다.
# 결과: A, B는 매칭되고 D는 x2가 NaN이 됨.
pd.merge(adf, bdf, how='right', on='x1')

In [ ]:
# 5. Inner Join (교집합)
# how='inner': 양쪽 모두에 존재하는 키(x1)만 남깁니다. (기본값)
# 결과: 둘 다 가지고 있는 'A', 'B'만 남고 C와 D는 사라집니다.
pd.merge(adf, bdf, how='inner', on='x1')

In [ ]:
# 6. Outer Join (합집합)
# how='outer': 양쪽의 모든 키를 합칩니다. 빈 곳은 NaN으로 채웁니다.
# 결과: A, B, C, D 모든 행이 살아납니다.
pd.merge(adf, bdf, how='outer', on='x1')

3. 필터링을 이용한 데이터 추출
merge를 쓰지 않고도, isin 함수를 이용해 특정 데이터가 다른 쪽에 포함되어 있는지 확인할 수 있습니다.

In [ ]:
# 7. 포함 여부 확인 (isin)
# adf의 x1 값들이 bdf의 x1에 들어있는지 확인합니다.
# 결과: A(True), B(True), C(False)
adf.x1.isin(bdf.x1)

In [ ]:
# 8. 공통된 데이터만 추출하기
# 위의 True인 행들만 뽑아내면 Inner Join과 비슷한 효과를 냅니다.
# 결과: A, B 행 출력
adf[adf.x1.isin(bdf.x1)]

In [ ]:
# 9. 공통되지 않은(왼쪽에만 있는) 데이터 추출하기
# 물결표시(~)는 NOT을 의미합니다. 즉, bdf에 없는 adf 데이터만 찾습니다.
# 결과: C 행 출력 (Left Join 후 NaN인 것 찾기와 유사)
adf[~adf.x1.isin(bdf.x1)]

4. 고급 병합 기법 (Indicator 활용)
새로운 예제(ydf, zdf)를 만들어, 데이터가 어느 쪽 테이블에서 왔는지(Left/Right/Both) 표시하는 indicator 기능을 사용해 봅니다. 이는 두 데이터셋의 차이점(Diff)을 찾을 때 매우 유용합니다.

In [ ]:
# 실습 데이터 생성
ydf = pd.DataFrame({"x1": ["A", "B", "C"], "x2": [1, 2, 3]})
ydf

In [ ]:
# 10. 기본 병합
# 공통 컬럼(x1, x2)을 기준으로 자동으로 Inner Join 합니다.
zdf = pd.DataFrame({"x1": ["B", "C", "D"], "x2": [2, 3, 4]})
zdf

In [ ]:
# 11. Outer Join
# 모든 데이터를 합칩니다.
pd.merge(ydf, zdf)

In [ ]:
# 12. Indicator(출처 표시) 기능을 활용한 차집합 구하기 (중요!)
# indicator=True: '_merge'라는 컬럼이 생겨서 데이터의 출처(left_only, right_only, both)를 알려줍니다.
# .query(...): 조건에 맞는 행만 걸러냅니다.
# .drop(...): 불필요해진 _merge 컬럼을 삭제합니다.
pd.merge(ydf, zdf, how='outer')

In [ ]:
# 12. Indicator(출처 표시) 기능을 활용한 차집합 구하기 (중요!)
# indicator=True: '_merge'라는 컬럼이 생겨서 데이터의 출처(left_only, right_only, both)를 알려줍니다.
# .query(...): 조건에 맞는 행만 걸러냅니다.
# .drop(...): 불필요해진 _merge 컬럼을 삭제합니다.
pd.merge(ydf, zdf, how='outer', indicator=True
        ).query('_merge == "left_only"'
               ).drop(columns=['_merge'])